#### FASE 0: CONFIGURACIÓN

In [ ]:
# Librerías

import pandas as pd 
import numpy as np 

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import KNNImputer


# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt


# Evaluar linealidad de las relaciones entre las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats
from scipy.stats import shapiro, poisson, chisquare, expon, kstest
from scipy.stats import levene, bartlett, shapiro

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore") # Para evitar errores en el uso de palette en seaborn


# ver todas las columnas
pd.set_option('display.max_columns', None)
# ver todas las filas
pd.set_option('display.max_rows', None)

In [ ]:
# Función EDA

def basic_eda(df):

    print('🌷Ejemplo de datos del DF:')
    display(df.head(3))
    display(df.tail(3))
    display(df.sample(3))
    print('________________________________________________________________________________________________________')

    print('🌻Número de Filas:')
    display(df.shape[0])
    print('________________________________________________________________________________________________________')

    print('🌱Número de Columnas:')
    display(df.shape[1])
    print('________________________________________________________________________________________________________')

    print('🌼Información de la tabla:')
    display(df.info())
    print('________________________________________________________________________________________________________')

    print('🌑Nombre de las columnas:')
    display(df.columns)
    print('________________________________________________________________________________________________________')

    print('🍄Descripción de los datos numéricos:')
    display(df.describe().T)
    print('________________________________________________________________________________________________________')

    print('🌋Descripción de los datos no-numéricos:')
    try:
        display(df.describe(include='object').T)
    except:
        pass
    print('________________________________________________________________________________________________________')

    print('🍂Saber si hay datos únicos:')
    display(df.nunique())
    print('________________________________________________________________________________________________________')

    print('🐖Que datos son nulos por columnas:')
    display(df.isnull().sum())
    print('________________________________________________________________________________________________________')

    print('🐲Filas duplicadas:')
    total_duplicados = df.duplicated().sum()
    if total_duplicados > 0:
        print(f'cantidad de duplicados: {total_duplicados}')
        print('Primeros duplicados')
        display(df[df.duplicated()].head(3))
    else:
        print('No hay duplicados')
    print('________________________________________________________________________________________________________')

    print('🪹 Columnas constantes (solo 1 valor único):')
    constantes = df.columns[df.nunique() <= 1]
    if len(constantes) > 0:
        print(f'{len(constantes)} columnas con 1 valor único:')
        display(constantes)
    else:
        print('No hay columnas constantes')
    print('________________________________________________________________________________________________________')
    
    print('🚀 Valores únicos en columnas categóricas:')
    for col in df.select_dtypes(include='object'):
        print(f'🔸 {col}')
        print('-----------------------------')
        print(df[col].unique())
        print('________________________________________________________________________________________________________')

    print('🧬 Tipos de datos por columna:')
    display(df.dtypes.value_counts())
    print('________________________________________________________________________________________________________')

In [ ]:
# Función unión de tablas con el método "right"

def to_union(df1,df2):
    df_new = df1.merge(df2, how='right')
    return df_new

In [ ]:
# Función para localizar y eliminar los registros duplicados

def duplicates(df):
    if df.duplicated().sum() > 0:
        df= df.drop_duplicates()
        return df
    else:
        return df

In [ ]:
# Función para cambiar el tipo de dato de una columna según necesidad

def change_data(df, column, type):
    try:
        if type == "int":
            df[column] = df[column].astype(int)
        elif type == "object":
            df[column] = df[column].astype(str)
        elif type == "datetime":
            df[column] = pd.to_datetime(df[column], errors='coerce')
        elif type == "float":
            df[column] = df[column].astype(float)
        else:
            print("Solo se acepta: int, object, datetime o float")
    except:
        print("Hay nulos")

In [ ]:
# Función para concatenar las fechas en una sola columna y eliminar las originales

def date_union (df, year_column, month_column, date_column): 
    localizacion = df.columns.get_loc(year_column)
    df.insert(localizacion, date_column, None)
    df[date_column] = df[year_column].astype(str) + "-" + df[month_column].astype(str).str.zfill(2) + "-01"
    df[date_column] = pd.to_datetime(df[date_column])
    df[date_column] = df[date_column].dt.date
    df.drop([year_column, month_column], axis=1, inplace=True)

In [ ]:
# Función para realizar una prueba de hipótesis con múltiples grupos

def multigroup_hypothesis_test(*args):

    """
    Realiza una prueba de hipótesis para comparar grupos.
    1. Primero verifica si los datos son normales usando el test de Shapiro-Wilk o Kolmogorov-Smirnov.
    2. Si los datos son normales, usa Bartlett para probar igualdad de varianzas. Si no son normales, usa Levene.
    3. Si las varianzas son iguales, usa el ANOVA; si no, usa la versión de Kruskal-Wallis.

    Parámetros:
    *args: listas o arrays con los datos de cada grupo. Espera VARIOS grupos a comparar

    Retorna:
    dict con resultados del test de normalidad, varianza e hipótesis.
    """


    if len(args) < 2:
        raise ValueError("Se necesitan al menos dos conjuntos de datos para realizar la prueba.")

    # Test de normalidad por grupo
    normality = []
    for group in args:
        if len(group) > 50:
            p_value_norm = stats.kstest(group, 'norm').pvalue
        else:
            p_value_norm = stats.shapiro(group).pvalue
        normality.append(p_value_norm > 0.05)

    normal_data = all(normality)

    # Test de igualdad de varianzas para todos los grupos juntos
    if normal_data:
        p_variance_value = stats.bartlett(*args).pvalue
    else:
        p_variance_value = stats.levene(*args, center="median").pvalue

    equal_variances = p_variance_value > 0.05

    # Test final según normalidad y varianzas
    if normal_data and equal_variances:
        t_stat, p_value = stats.f_oneway(*args)
        test_used = "ANOVA"
    else:
        t_stat, p_value = stats.kruskal(*args)
        test_used = "Kruskal-Wallis"

    alfa = 0.05

    result = {
        "Test de Normalidad": normality,
        "Datos Normales": normal_data,
        "p-valor Varianza": p_variance_value,
        "Varianzas Iguales": equal_variances,
        "Test Usado": test_used,
        "Estadístico": t_stat,
        "p-valor": p_value,
        "Conclusión": "Rechazamos H0. Hay diferencias significativas entre grupos" if p_value < alfa else "No se rechaza H0. No hay diferencias significativas entre grupos"
    }

    print("\n📊 **Resultados de la Prueba de Hipótesis Multigrupos** 📊")
    print(f"✅ Normalidad en todos los grupos: {'Sí' if normal_data else 'No'}")
    print(f"   - Normalidad por grupo: {normality}")
    print(f"✅ Varianzas: {'Iguales' if equal_variances else 'Desiguales'} (p = {p_variance_value:.4f})")
    print(f"✅ Test aplicado: {test_used}")
    print(f"📉 Estadístico: {t_stat:.4f}, p-valor: {p_value:.4f}")
    print(f"🔍 Conclusión: {result['Conclusión']}\n")

In [ ]:
# Establecemos paleta de color para los gráficos 

pastel_colors = [
    "#A8DADC",  # Verde azulado pastel (frío)
    "#FFE5B4",  # Amarillo pastel (cálido)
    "#F1C0E8",  # Lila suave (frío)
    "#FFD6D6",  # Rosa melocotón muy suave (cálido)
    "#B5EAEA",  # Turquesa pastel (frío)
    "#FFCBC1",  # Salmón pastel (cálido)
    "#C1E1C1",  # Verde menta claro (frío)
    "#FFE3E3",  # Rosa claro (cálido)
    "#D4A5A5",  # Marrón claro pastel (cálido)
    "#C6D8D3",  # Verde grisáceo pastel (frío)
    "#F7E6C4",  # Beige claro (cálido)
    "#B9AEDC",  # Lavanda pastel (frío)
]

In [ ]:
# Configuramos valores para el formato de las tablas por defecto:

plt.rcParams.update({
    'font.family': 'Comic Sans MS',
    'axes.titlesize': 18,
    'axes.titleweight': 'bold',
    'axes.labelsize': 14,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'figure.facecolor' : 'lightgray',
    'axes.facecolor': 'darkgray'
})

#### FASE 1: EXPLORACIÓN Y LIMPIEZA

In [ ]:
# Abrimos el archivo 1

df_fligh = pd.read_csv("Customer Flight Activity.csv")
df_fligh.sample(5)

In [ ]:
# Abrimos el archivo 2

df_loyalty=pd.read_csv("Customer Loyalty History.csv")
df_loyalty.sample(5)

In [ ]:
# Aplicamos la función de EDA al archivo 1 para entender los datos

basic_eda(df_fligh)

In [ ]:
# Aplicamos la función de EDA al archivo 2 para entender los datos

basic_eda(df_loyalty)

In [ ]:
# Tras comprobar que hay filas duplicadas en el archivo 1 usamos la función para eliminar duplicados

df_fligh = duplicates(df_fligh)

# Comprobamos también que la información del archivo 1 continene los vuelos reservados de los clientes. Si la columna de vuelos totales es 0 no aporta valor. Eliminamos las filas cuya columna es 0:

df_fligh = df_fligh[df_fligh["Total Flights"] != 0]

# Unimos las dos tablas en una sola por el método right teniendo en cuenta que el archivo 2 tiene solo un registro por cliente y el archivo 1 tiene varios registros por cliente_

df = to_union(df_loyalty, df_fligh)

# Practicamos un nuevo análisis EDA sobre el archivo unificado: 

basic_eda(df)

In [ ]:
# Generamos histogramas para valorar las columnas numéricas

df.select_dtypes(include=[np.number]).hist(figsize=(16,12), bins=30, color = "black")
plt.tight_layout()
plt.show()

In [ ]:
# Comprobamos que la columna Salary tiene valores negativos. Utilizamos .abs() para sacar el valor absoluto:

df["Salary"] = df["Salary"].abs()

# También tiene valores nulos. Elegimos el método KNN para la imputación y facilitamos otras columnas numéricas en las que apoyar la imputación: 

columns_to_imput = ["Salary", "CLV", "Distance", "Points Accumulated"]
imputer_knn = KNNImputer(n_neighbors=5)
df[columns_to_imput] = imputer_knn.fit_transform(df[columns_to_imput])

# Completamos los nulos de las columnas de mes y año de cancelación a fechas absurdas en el futuro: 

df["Cancellation Year"] = df["Cancellation Year"].fillna(2262)
df["Cancellation Month"] = df["Cancellation Month"].fillna(1)

# Una vez gestionado los nulos, cambiamos el tipo de dato de estas columnas de float a int: 

columns_int = ["Points Accumulated", "Salary", "Cancellation Year", "Cancellation Month"]

for column in columns_int: 
    change_data(df, column, "int")
    print(f"Cambiada la columna {column}")

# Concatenamos las fechas en una sola columna juntando el año, el mes y el día 1. Lo convertimos a datatime y eliminamos las columnas de año y mes iniciales con la función date_union: 

date_union(df, "Enrollment Year", "Enrollment Month", "Enrollment Date")
date_union(df, "Cancellation Year", "Cancellation Month", "Cancellation Date")
date_union(df, "Year", "Month", "Date")

In [ ]:
df.sample(5)

In [ ]:
# Guardamos en un nuevo csv los datos limpios para empezar con los ejercicios. 

df.to_csv("Customer_union.csv")

In [ ]:
df=pd.read_csv("Customer_union.csv", index_col = 0)
df.sample(5)

#### FASE 2: VISUALIZACIÓN  
  
- 1. ¿Cómo se distribuye la cantidad de vuelos reservados por mes durante el año?

In [ ]:
# Gráfico para ejercicio 1

df_booking = df.groupby("Date")["Flights Booked"].sum()
df_booking.index = pd.to_datetime(df_booking.index)
formatted_dates = df_booking.index.strftime("%B %Y")  # Da formato a la columna para mostrar mes en letra

plt.figure(figsize=(12, 6))
plt.bar(x=formatted_dates, height=df_booking.values, color=pastel_colors, edgecolor='black')

plt.xlabel('Meses')
plt.ylabel('Vuelos Reservados')
plt.title('Vuelos Reservados por Mes', fontsize=22)
plt.xticks(rotation=45)
ticks_y = np.arange(0, 110000, 10000)
plt.yticks(ticks_y)
plt.grid(True, axis='y', color='#eeeeee')
plt.gca().spines[['right', "top"]].set_visible(False) # quitamos la línea de arriba y de la derecha
plt.axvline(x=12 - 0.5, color='gray', linestyle='--', linewidth=1)
plt.tight_layout()
plt.show()

Respuesta: tras poder comprobar la información por mes de dos años consecutivos vemos que el patrón se repite, siendo el mes con más movimiento el de julio pero seguido de cerca por los meses de junio, agosto y diciembre. Podemos relacionar estos picos con las vacaciones estivales y Navidad. Por contra, los meses con menos movimiento son enero y febrero que coincide con que no hay fiestas reseñables en estos meses. 

- 2. ¿Existe una relación entre la distancia de los vuelos y los puntos acumulados por los cliente?

In [ ]:
# Gráfico para ejercicio 2

fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (20, 5))

sns.regplot(x = "Distance", 
            y = "Points Accumulated", 
            data = df, 
            marker = "d", 
            line_kws = {"color": "black", "linewidth": 1}, # cambiamos el color y el grosor de la linea de tendencia
            scatter_kws = {"color": "crimson", "s": 2}, # cambiamos el color y el tamaño de los puntos del scaterplot
            ax = axes[0])
axes[0].set_xlabel("Distancia")
axes[0].set_ylabel("Puntos Acumulados")
axes[0].set_title("Relación distancia recorrida y puntos acumulados")
axes[0].spines[['top', 'right']].set_visible(False)

df_corr = df[["Distance", "Points Accumulated"]].corr( method = "spearman").round(2)
df_corr.index = ['Distancia', 'Puntos']
df_corr.columns = ['Distancia', 'Puntos']
sns.heatmap(df_corr, 
            cmap='coolwarm', 
            annot=True, 
            fmt='.2f', 
            linewidths=0.5, 
            vmin=-1, 
            vmax=1, 
            ax = axes[1])
axes[1].set_title("Matriz de Correlación de distancia recorrida y puntos acumulados", fontsize=14)
;

Respuesta: definitivamente sí, están estrechamente relacioneados. A más distancia recorrida en vuelo más puntos acumulan los clientes. 

- 3. ¿Cuál es la distribución de los clientes por provincia o estado?

In [ ]:
# Gráfico para ejercicio 3

plt.figure(figsize=(8, 4))

sns.countplot(data=df, 
              x="Province", 
              order=df["Province"].value_counts().index, 
              palette=pastel_colors, 
              edgecolor='black', 
              width=0.6)
plt.title('Distribución de clientes por provincia')
plt.xticks(rotation=45)
plt.xlabel('Provincias')
plt.ylabel('Número de clientes')
plt.grid(True, axis='y', color='#eeeeee')
plt.gca().spines[['right', "top"]].set_visible(False) # quitamos la línea de arriba y de la derecha
plt.show()
;

Respuesta: Esta es la distribución, siendo con diferencia Ontario, British Columbia y Quebec las que acumulan la mayor parte de los clientes. 

- 4. ¿Cómo se compara el salario promedio entre los diferentes niveles educativos de los clientes?

In [ ]:
# Gráfico para ejercicio 4

plt.figure(figsize=(8, 4))

df_salary = df.groupby("Education")["Salary"].mean()
order_education = ['High School or Below', 'College', 'Bachelor', 'Master', 'Doctor']
df_salary = df_salary.reindex(order_education)
df_salary.index = ['Secundaria', 'Técnica', 'Grado', 'Máster', 'Doctorado']

df_salary.plot(kind='bar', color = pastel_colors, edgecolor='black', width=0.4)
plt.title('Salario promedio por nivel de estudios')
plt.xticks(rotation=45)
plt.xlabel('Nivel de educación')
plt.ylabel('Salario promedio')
plt.grid(True, axis='y', color='#eeeeee')
plt.gca().spines[['right', "top"]].set_visible(False) # quitamos la línea de arriba y de la derecha
plt.show()
;

Respuesta: el salario aumenta en relación al nivel de estudios con la única excepción de que los técnicos cobran de media ligeramente más que las personas con grado universitario. 

- 5. ¿Cuál es la proporción de clientes con diferentes tipos de tarjetas de fidelidad?

In [ ]:
# Gráfico para ejercicio 5

plt.figure(figsize=(7, 4))

df_fidelity = df["Loyalty Card"].value_counts()
plt.pie(df_fidelity.values, 
        labels= df_fidelity.index,
        data = df, 
        autopct=  '%1.1f%%', 
        colors = pastel_colors, 
        textprops={'fontsize': 10}, 
        startangle=90, 
        wedgeprops={'edgecolor': 'black'} );
plt.title('Proporción clientes por tarjeta de fidelidad',fontsize=18, fontname='Comic Sans MS')
plt.axis('equal')  # Para que el círculo sea perfecto
plt.show()
;

Respuesta: nada que añadir a lo reflejado en el gráfico:  
  
Star: 45.6%  
Aurora: 20.6%  
Nova: 33.8%

- 6. ¿Cómo se distribuyen los clientes según su estado civil y género?

In [ ]:
# Gráfico para ejercicio 6

plt.figure(figsize=(7, 5))

new_names = ["Casado", "Soltero", "Divorciado"]
sns.countplot(x = "Marital Status", 
              data = df,
              palette = pastel_colors,
              width=0.4, 
              hue = "Gender", 
              edgecolor='black')
plt.xlabel("Estado civil")
plt.ylabel("Nº clientes")
plt.title('Distribución de clientes por género y estado civil')
plt.xticks(ticks=range(len(new_names)), labels=new_names, rotation=0)
plt.grid(True, axis='y', color='#eeeeee')
plt.legend(title="Género", labels=["Mujeres", "Hombres"])
plt.gca().spines[['right', "top"]].set_visible(False) # quitamos la línea de arriba y de la derecha
plt.show()
;

Respuesta: no hay diferencias reseñables entre género pero sí entre estado civil. Son lso casados los que más reservan seguidos de los solteros y en último caso los divorciados. 

#### BONUS

- Preparación de Datos: Filtra el conjunto de datos para incluir únicamente las columnas relevantes.   

- Análisis Descriptivo: Agrupa los datos por nivel educativo y calcula estadísticas descriptivas básicas (como el promedio, la desviación estándar) del número de vuelos eservados para cada grupo.  
  
- Prueba Estadística: Realiza una prueba de hipótesis para determinar si existe una diferencia significativa en el número de vuelos reservados entre los diferentes niveles educativos.

In [ ]:
#  Preparación de datos: Generamos un nuevo df con las columnas indicadas: 

df_bon = df[["Flights Booked", "Education"]]

#  Análisis descriptivo: Agrupamos para visualizar las variables por nivel de educación: 

df_bonus = df_bon.groupby("Education")["Flights Booked"].describe().T
display(df_bonus)

# Generamos una variable por cada nivel de educación y mostramos sólo la columna de vuelos reservados: 

bachelor_group = df_bon[df_bon["Education"] == "Bachelor"]["Flights Booked"]
college_group = df_bon[df_bon["Education"] == "College"]["Flights Booked"]
doctor_group = df_bon[df_bon["Education"] == "Doctor"]["Flights Booked"]
secondary_group = df_bon[df_bon["Education"] == "High School or Below"]["Flights Booked"]
master_group = df_bon[df_bon["Education"] == "Master"]["Flights Booked"]

# Prueba estadística: Activamos la función de la prueba de hipótesis adaptada para gestionar con varios grupos:

multigroup_hypothesis_test(bachelor_group, college_group, doctor_group, secondary_group, master_group)

Respuesta : los niveles educativos analizados no muestran diferencias significativas en la cantidad promedio de vuelos reservados, según los datos y la prueba no paramétrica aplicada.